# TensorFlow basic

Import numpy and tensorflow

In [1]:
import numpy as np
import tensorflow as tf

Reset default graph to avoid duplicated nodes.

In [2]:
tf.reset_default_graph()

Create a graph representing $ f = x^2 y + y + 2$. Here the initial values are 3 for x and 4 for y.

In [3]:
x = tf.Variable(initial_value=3, dtype=tf.float32, name='x')
y = tf.Variable(initial_value=4, dtype=tf.float32, name='y')
f = x**2*y + y + 2

Evaluate this graph and print out the value of f

In [4]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(f))

42.0


Create variable x1 with initial value 1 in the default graph. Verify that x1 is in the default graph.

In [5]:
x1 = tf.Variable(initial_value=1.0, name='x1')

In [6]:
x1.graph is tf.get_default_graph()

True

Creata a new graph called "newGraph". Temporarily set newGraph as the default and then create variable x2 with initial value 2 in newGraph. Verify that x2 is in newGraph, not in the default graph.

In [7]:
newGraph = tf.Graph()

In [8]:
with newGraph.as_default():
    x2 = tf.Variable(initial_value=1.0, name='x2')

In [9]:
x2.graph is newGraph

True

In [10]:
x2.graph is tf.get_default_graph()

False

# Linear regression

Load california housing data from sklearn and return the result as housing.

In [11]:
from sklearn.datasets import california_housing

In [12]:
housing = california_housing.fetch_california_housing()

Normalize the predictor and add bias term to it and return it as X_training. Return the target as y_training

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
scalar = StandardScaler()
X_training = scalar.fit_transform(housing.data)
(m, n) = X_training.shape
X_training = np.c_[np.ones((m, 1)), X_training]

In [15]:
y_training = housing.target.reshape((-1, 1))

In order to visualize the training, define log file tf_logs/run-time, where time is a string of UTC time which varies

In [16]:
from datetime import datetime

In [17]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

In [18]:
logdir = 'tf_logs/run_{}'.format(now)

Define a linear regression model y_pred = X * theta

In [19]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [20]:
with tf.name_scope('model') as model:
    theta = tf.Variable(tf.random_uniform((n+1, 1), -1, 1), dtype=tf.float32, name='theta')
    y_pred = tf.matmul(X, theta, name='y_pred')

Define the mse error

In [21]:
with tf.name_scope('error') as error:
    mse = tf.reduce_mean(tf.square(y - y_pred), name='mse')

Define a momentum optimizer and a training op to minimize mse

In [22]:
with tf.name_scope('training') as training:
    optimizer = tf.train.MomentumOptimizer(learning_rate=0.0001, momentum=0.9)
    training_op = optimizer.minimize(mse)

In order to visualize, define a node of mse_summary which would convert mse value to a scalar string and also give mse the name of 'MSE' in TensorBoard

In [23]:
mse_summary = tf.summary.scalar('MSE', mse)

In order to visualize, also define a FileWriter that writes summaries to logfiles in the log directory.

In [24]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

Define a batch size of 100. Then define a function fetch_batch(batch_size), which randomly pick up batch_size samples from training data, and return X_batch and y_batch

In [25]:
batch_size = 100
def fetch_batch(batch_size=batch_size):
    idx = np.random.randint(0, m, size=batch_size)
    X_batch = X_training[idx]
    y_batch = y_training[idx]
    return (X_batch, y_batch)

Train the model by 20 epoches, and output the final theta. Also define a saver to save the model every 2 epoches in tmp folder. Also save the final model in tmp folder. Evaluate mse_summary node every 10 mini-batches and save (mse value, step value) by FileWriter for visualization.

In [26]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_batch = int(np.ceil(m/batch_size))

In [27]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(20):
        if epoch % 2 == 0:
            saver.save(sess, 'tmp/linear_model.ckpt')
        for batch in range(n_batch):
            X_batch, y_batch = fetch_batch(batch_size)
            if batch % 10 == 0:
                summary_str = sess.run(mse_summary, feed_dict={X:X_batch, y:y_batch})
                step = n_batch * epoch + batch
                file_writer.add_summary(summary_str, global_step=step)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    saver.save(sess, 'tmp/final_liner_model.ckpt')
    file_writer.close()

Now visualize the graph and training curves in TensorBoard